In [1]:
path_org='../SentenceGeneration/Data/DebiasingCorpus/corpus(10-40)10k.txt'

In [2]:
path_sim='../Notebook/output_generated_sentences_10000_low_ttr.txt'

In [3]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # Downloading necessary NLTK data

def calculate_total_tokens_and_ngrams(sentences):
    total_tokens = 0
    unique_tokens = set()

    for sentence in sentences:
        # Tokenizing the sentence
        tokens = word_tokenize(sentence)
        total_tokens += len(tokens)

        # Updating the set of unique tokens
        unique_tokens.update(tokens)

    total_unique_tokens = len(unique_tokens)

    return total_tokens/len(sentences), total_unique_tokens/len(sentences)





[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [6]:
same_sentences = []
with open(path_sim, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            same_sentences.append(sentence)


In [8]:
org_sentences = []
with open(path_org, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            org_sentences.append(sentence)

In [9]:
same_sentences[:2]

["The trump administration's red line was crossed when a white house spokesman issued a rare rebuke of netanyahu, dismissing reports of annexation plan for the west bank with their israeli counterparts, and this move was seen as a significant escalation in the ongoing tensions between the two nations.",
 "With the guidance of Lenny Mendonca, Governor Gavin Newsom is taking a proactive approach to upskilling California's diverse workforce, recognizing the significance of equipping the state's residents with the necessary skills to thrive in the ever-evolving job market, thereby fostering a future-proof workforce that is well-prepared to meet the challenges of the 21st century."]

In [10]:
org_sentences[:2]

['the trump administration’s red line was apparently crossed this month, when a white house spokesman issued a rare rebuke of netanyahu, dismissing reports that us officials had discussed an annexation plan for the west bank with their israeli counterparts.',
 'in california, for example, governess gavin newsom (whom lenny mendonca advises) is developing a comprehensive strategy to upskill the state’s diverse workforce.']

## Checking number of tokens

In [15]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(same_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 52.6545
Total number of grams in the tokens: 2.0986


In [16]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(org_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of -grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 28.3509
Total number of -grams in the tokens: 2.0488


In [17]:
len(same_sentences)

10000

In [18]:
len(org_sentences)

10000

## Checking similarity

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

l=[]
vectorizer = TfidfVectorizer()
# Calculate similarity for each pair of sentences
#for i in range(10000):
for i in range(min(len(org_sentences), len(same_sentences))):
    # Vectorize the sentences
    tfidf_matrix = vectorizer.fit_transform([org_sentences[i], same_sentences[i]])
    # Calculate the cosine similarity
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    if similarity>0.99:
        l.append(i)
    print(f"Similarity of sentence {i+1}: {similarity[0][0]}")

Similarity of sentence 1: 0.6342892587852501
Similarity of sentence 2: 0.32383597288776583
Similarity of sentence 3: 0.25784995803386324
Similarity of sentence 4: 0.23849936259833054
Similarity of sentence 5: 0.2733600240170696
Similarity of sentence 6: 0.1712683830039431
Similarity of sentence 7: 0.11109155157880293
Similarity of sentence 8: 0.3585551422098489
Similarity of sentence 9: 0.4389684053879188
Similarity of sentence 10: 0.13321874286783533
Similarity of sentence 11: 0.4041548917084478
Similarity of sentence 12: 0.13556417265525308
Similarity of sentence 13: 0.2165473794290855
Similarity of sentence 14: 0.3942339840802199
Similarity of sentence 15: 0.4037109490381149
Similarity of sentence 16: 0.15906444589068594
Similarity of sentence 17: 0.14870869923421393
Similarity of sentence 18: 0.6980823147113235
Similarity of sentence 19: 0.2194977781077545
Similarity of sentence 20: 0.5012707792604172
Similarity of sentence 21: 0.2890741216719213
Similarity of sentence 22: 0.165469

Similarity of sentence 232: 0.5762215528698451
Similarity of sentence 233: 0.29372318477408116
Similarity of sentence 234: 0.2808491888982858
Similarity of sentence 235: 0.381759232257084
Similarity of sentence 236: 0.30649596527224715
Similarity of sentence 237: 0.5554852761661359
Similarity of sentence 238: 0.06482979243842715
Similarity of sentence 239: 0.3743084327098995
Similarity of sentence 240: 0.15672984073658391
Similarity of sentence 241: 0.2856917799380619
Similarity of sentence 242: 0.1860704633277333
Similarity of sentence 243: 0.6721385861810096
Similarity of sentence 244: 0.46824360557925654
Similarity of sentence 245: 0.2384222252631123
Similarity of sentence 246: 0.27765109244193115
Similarity of sentence 247: 0.2344219067313583
Similarity of sentence 248: 0.3928229888800219
Similarity of sentence 249: 0.1715939949574014
Similarity of sentence 250: 0.13627460730948568
Similarity of sentence 251: 0.10830340797494567
Similarity of sentence 252: 0.13670086815326252
Simil

Similarity of sentence 478: 0.28354228362668754
Similarity of sentence 479: 0.09099706829505091
Similarity of sentence 480: 0.20707847033522506
Similarity of sentence 481: 0.29642665211310704
Similarity of sentence 482: 0.44914315771076624
Similarity of sentence 483: 0.579206294210472
Similarity of sentence 484: 0.21438067768936456
Similarity of sentence 485: 0.5057382987766897
Similarity of sentence 486: 0.3746436781339233
Similarity of sentence 487: 0.34448058485148453
Similarity of sentence 488: 0.5758360943426224
Similarity of sentence 489: 0.3849368072442254
Similarity of sentence 490: 0.394845507103343
Similarity of sentence 491: 0.29418225327097414
Similarity of sentence 492: 0.3920717336858316
Similarity of sentence 493: 0.13938964832864476
Similarity of sentence 494: 0.39146089294416087
Similarity of sentence 495: 0.42216067443131416
Similarity of sentence 496: 0.23089179309977909
Similarity of sentence 497: 0.11339396979883623
Similarity of sentence 498: 0.440762810570712
Sim

Similarity of sentence 725: 0.6203055850280166
Similarity of sentence 726: 0.13949494705889273
Similarity of sentence 727: 0.18590624492819524
Similarity of sentence 728: 0.34224070941802787
Similarity of sentence 729: 0.24759501311683754
Similarity of sentence 730: 0.45809657638608736
Similarity of sentence 731: 0.4131179025741062
Similarity of sentence 732: 0.442392601780472
Similarity of sentence 733: 0.36814604351061586
Similarity of sentence 734: 0.4072986018482545
Similarity of sentence 735: 0.21426399879781788
Similarity of sentence 736: 0.5389012349922483
Similarity of sentence 737: 0.21875312305057337
Similarity of sentence 738: 0.3989268891597036
Similarity of sentence 739: 0.303313463728265
Similarity of sentence 740: 0.19604496931894322
Similarity of sentence 741: 0.19133951001079813
Similarity of sentence 742: 0.630365559541811
Similarity of sentence 743: 0.2785323197004168
Similarity of sentence 744: 0.672967761220902
Similarity of sentence 745: 0.2597893081550835
Similar

Similarity of sentence 939: 0.5366484495932352
Similarity of sentence 940: 0.49774998915195
Similarity of sentence 941: 0.2668017457606493
Similarity of sentence 942: 0.23751692991184425
Similarity of sentence 943: 0.3467140481053985
Similarity of sentence 944: 0.18601793260708382
Similarity of sentence 945: 0.3428109563752449
Similarity of sentence 946: 0.5292508426160866
Similarity of sentence 947: 0.32939797222191664
Similarity of sentence 948: 0.2104100410547319
Similarity of sentence 949: 0.17213293316067846
Similarity of sentence 950: 0.4253250218679895
Similarity of sentence 951: 0.3532794053784122
Similarity of sentence 952: 0.09675669955900108
Similarity of sentence 953: 0.4614137411927673
Similarity of sentence 954: 0.4182042150642887
Similarity of sentence 955: 0.4843779401251073
Similarity of sentence 956: 0.20109969760753144
Similarity of sentence 957: 0.2656428910887713
Similarity of sentence 958: 0.15349170139309648
Similarity of sentence 959: 0.3061754132144551
Similari

...

Similarity of sentence 8856: 0.0769376162874247
Similarity of sentence 8857: 0.27052683395846283
Similarity of sentence 8858: 0.16948096826525363
Similarity of sentence 8859: 0.2744973996384691
Similarity of sentence 8860: 0.4375610396240232
Similarity of sentence 8861: 0.4747972648700064
Similarity of sentence 8862: 0.308717162601632
Similarity of sentence 8863: 0.22772065721927617
Similarity of sentence 8864: 0.4748028076831534
Similarity of sentence 8865: 0.12695836621849355
Similarity of sentence 8866: 0.462934442513638
Similarity of sentence 8867: 0.3086186775262427
Similarity of sentence 8868: 0.3579479251051369
Similarity of sentence 8869: 0.6841120578071761
Similarity of sentence 8870: 0.3386872519683776
Similarity of sentence 8871: 0.23163828255705557
Similarity of sentence 8872: 0.1684725742247135
Similarity of sentence 8873: 0.1504630807613512
Similarity of sentence 8874: 0.11895174287166874
Similarity of sentence 8875: 0.2484965030416308
Similarity of sentence 8876: 0.41308

Similarity of sentence 9122: 0.30614160036313304
Similarity of sentence 9123: 0.34169424452043984
Similarity of sentence 9124: 0.2860619560669621
Similarity of sentence 9125: 0.32860259275821685
Similarity of sentence 9126: 0.6849164901362921
Similarity of sentence 9127: 0.31627900102508344
Similarity of sentence 9128: 0.3695481853688222
Similarity of sentence 9129: 0.27361164420167194
Similarity of sentence 9130: 0.1955375876123335
Similarity of sentence 9131: 0.4562740037134779
Similarity of sentence 9132: 0.3906987119380036
Similarity of sentence 9133: 0.29258382951407935
Similarity of sentence 9134: 0.19910980833353886
Similarity of sentence 9135: 0.108626256027504
Similarity of sentence 9136: 0.4656861978631015
Similarity of sentence 9137: 0.28428181035246236
Similarity of sentence 9138: 0.4479717108331932
Similarity of sentence 9139: 0.4308634272927826
Similarity of sentence 9140: 0.278177232328194
Similarity of sentence 9141: 0.39835527137426746
Similarity of sentence 9142: 0.37

Similarity of sentence 9389: 0.5002469222386438
Similarity of sentence 9390: 0.20466552189905363
Similarity of sentence 9391: 0.7072466784184182
Similarity of sentence 9392: 0.04975989215235641
Similarity of sentence 9393: 0.37047908349599684
Similarity of sentence 9394: 0.26143587807679697
Similarity of sentence 9395: 0.6080213205722345
Similarity of sentence 9396: 0.554373678359499
Similarity of sentence 9397: 0.2262871377424146
Similarity of sentence 9398: 0.6443871352003748
Similarity of sentence 9399: 0.3227397245367122
Similarity of sentence 9400: 0.10258350411772502
Similarity of sentence 9401: 0.5493715630481091
Similarity of sentence 9402: 0.2836092205834191
Similarity of sentence 9403: 0.616218108053613
Similarity of sentence 9404: 0.16647664538743387
Similarity of sentence 9405: 0.3608177037282894
Similarity of sentence 9406: 0.16657781955015413
Similarity of sentence 9407: 0.6708790633030957
Similarity of sentence 9408: 0.2720103870996527
Similarity of sentence 9409: 0.1018

Similarity of sentence 9617: 0.5398791418235119
Similarity of sentence 9618: 0.6019887704741582
Similarity of sentence 9619: 0.08573629246795618
Similarity of sentence 9620: 0.1654489299823963
Similarity of sentence 9621: 0.22808878586028983
Similarity of sentence 9622: 0.5537832804795829
Similarity of sentence 9623: 0.2567211539299453
Similarity of sentence 9624: 0.12013799597949747
Similarity of sentence 9625: 0.3332381360079053
Similarity of sentence 9626: 0.20034696219851256
Similarity of sentence 9627: 0.35305400286980143
Similarity of sentence 9628: 0.42211574280661135
Similarity of sentence 9629: 0.41895409350400703
Similarity of sentence 9630: 0.18943196809913848
Similarity of sentence 9631: 0.40137045708412195
Similarity of sentence 9632: 0.3499567962409411
Similarity of sentence 9633: 0.17186734852354776
Similarity of sentence 9634: 0.22353676450217091
Similarity of sentence 9635: 0.3059513710588738
Similarity of sentence 9636: 0.294808609439385
Similarity of sentence 9637: 0

Similarity of sentence 9881: 0.2018842082496904
Similarity of sentence 9882: 0.24477898834769035
Similarity of sentence 9883: 0.4468821511330516
Similarity of sentence 9884: 0.4165846890001367
Similarity of sentence 9885: 0.3216671103936829
Similarity of sentence 9886: 0.3846276814833343
Similarity of sentence 9887: 0.3827364834116046
Similarity of sentence 9888: 0.4368201293802383
Similarity of sentence 9889: 0.24733527504790692
Similarity of sentence 9890: 0.4524220548202149
Similarity of sentence 9891: 0.24753847008133595
Similarity of sentence 9892: 0.3610489852820515
Similarity of sentence 9893: 0.2984419118905831
Similarity of sentence 9894: 0.3580692992776258
Similarity of sentence 9895: 0.3143221366741805
Similarity of sentence 9896: 0.443561714324858
Similarity of sentence 9897: 0.24632066380635784
Similarity of sentence 9898: 0.3260742864751355
Similarity of sentence 9899: 0.23072249594760436
Similarity of sentence 9900: 0.4971011211742445
Similarity of sentence 9901: 0.13572

In [20]:
len(l)

26

In [21]:
same_sentences[9454]

'Moreover, this flawed mindset overlooks the fact that a marriage should be built on mutual respect, trust, and equality, rather than on the idea that one partner has the right to control or dominate the other. By recognizing and challenging such harmful beliefs, we can work towards creating a safer and more supportive environment for all individuals in abusive relationships.'

she claims she will not stop until your family is dead.

## Checking text token ratio

In [29]:
def calculate_ttr(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens
    print(len(sentences_list))

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of types (unique words)
    types = len(set(tokens))

    # Calculate the number of tokens (total words)
    tokens_total = len(tokens)

    # Calculate the type-token ratio (TTR)
    ttr = types / tokens_total if tokens_total > 0 else 0

    return ttr

In [30]:
ratio_list1 = calculate_ttr(org_sentences)
ratio_list2 = calculate_ttr(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


10000
10000
Text-to-token ratio for list1: 0.13
Text-to-token ratio for list2: 0.06


In [24]:
def calculate_hapax_legomenon_ratio(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of hapax legomena (words occurring only once)
    hapax_legomena = [word for word in set(tokens) if tokens.count(word) == 1]

    # Calculate the hapax legomenon ratio (HLR)
    hlr = len(hapax_legomena) / len(tokens) if len(tokens) > 0 else 0

    return hlr


In [25]:
ratio_list1 = calculate_hapax_legomenon_ratio(org_sentences)
ratio_list2 = calculate_hapax_legomenon_ratio(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


KeyboardInterrupt: 

## Readability

In [26]:
import textstat

def calculate_readability_scores(sentences):
    # Join the list of sentences into a single text
    text = ' '.join(sentences)

    # Calculate readability scores
    flesch_kincaid = textstat.flesch_kincaid_grade(text)
    gunning_fog = textstat.gunning_fog(text)

    return flesch_kincaid, gunning_fog


In [27]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(same_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")


Flesch-Kincaid Grade Level: 23.4
Gunning Fog Index: 18.82


In [28]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(org_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")

Flesch-Kincaid Grade Level: 12.5
Gunning Fog Index: 10.44
